# Desafio 07

TNT

In [1]:
import pandas as pd
import numpy as np

# Carregando banco

In [2]:
dataset = pd.read_csv('tnt_database.csv')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9979 entries, 0 to 9978
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Açaí-Guaraná  9979 non-null   int64  
 1   Citrus        9979 non-null   int64  
 2   Estação       9979 non-null   object 
 3   LAT           9979 non-null   float64
 4   LONG          9979 non-null   float64
 5   Maçã-Verde    9979 non-null   int64  
 6   Movimentação  9979 non-null   int64  
 7   Original_269  9979 non-null   int64  
 8   Original_473  9979 non-null   int64  
 9   Pêssego       9979 non-null   int64  
 10  TARGET        9979 non-null   object 
 11  Tangerina     9979 non-null   int64  
 12  Tempo         9979 non-null   object 
 13  Zero          9979 non-null   int64  
 14  _id           9979 non-null   object 
 15  _rev          9979 non-null   object 
 16  row           9979 non-null   int64  
dtypes: float64(2), int64(10), object(5)
memory usage: 1.3+ MB


In [4]:
dataset.head()

,Açaí-Guaraná,Citrus,Estação,LAT,LONG,Maçã-Verde,Movimentação,Original_269,Original_473,Pêssego,TARGET,Tangerina,Tempo,Zero,_id,_rev,row
0,26,28,Pinheiros,-23.5664,-46.7030,43,78075,24,36,28,NORMAL,24,2019-6-23,13,0120472f15d821f24f91c6cf6701d88b,1-aa58175da90a12307d3ed70bdc902c2c,9173
1,41,9,Pinheiros,-23.5664,-46.7030,40,79432,53,78,41,NORMAL,38,2019-3-12,21,0120472f15d821f24f91c6cf670233ef,1-da709a8b94674992afdd9df7976f161d,9072
2,4,19,Butantã,-23.5844,-46.7252,25,67775,62,21,35,NORMAL,15,2018-4-7,51,0120472f15d821f24f91c6cf6704a6de,1-0ad2377710ecd310f8c810bfb2b04825,9457
3,19,43,Pinheiros,-23.5664,-46.7030,35,76773,29,61,18,NORMAL,37,2019-9-23,50,0120472f15d821f24f91c6cf67102d3a,1-c6e56500b4ad6860fa2396bd8453a7ba,9263
4,6,16,Pinheiros,-23.5664,-46.7030,26,77463,8,32,22,REABASTECER,10,2019-11-21,43,0120472f15d821f24f91c6cf6714cf5e,1-8cb748d7d25cb718673ce9ab4f5376f2,9321


In [25]:
dataset2 = dataset.drop(['LAT', 'LONG', '_id', '_rev', 'row', 'Estação', 'Tempo', 'Movimentação'], axis=1)


In [26]:
dataset2.head()

,Açaí-Guaraná,Citrus,Maçã-Verde,Original_269,Original_473,Pêssego,TARGET,Tangerina,Zero
0,26,28,43,24,36,28,NORMAL,24,13
1,41,9,40,53,78,41,NORMAL,38,21
2,4,19,25,62,21,35,NORMAL,15,51
3,19,43,35,29,61,18,NORMAL,37,50
4,6,16,26,8,32,22,REABASTECER,10,43


# Separando banco de treino e de teste

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit

In [28]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [29]:
for train_index, test_index in split.split(dataset2, dataset2.TARGET):
    data_train = dataset2.loc[train_index]    
    data_val = dataset2.loc[test_index]

In [30]:

print(data_train.shape)
print(data_val.shape)

(7983, 9)
(1996, 9)


## Seperando as features e target

In [31]:
x_train = data_train.drop(['TARGET'], axis=1)
y_train = data_train.TARGET

In [35]:
x_val = data_val.drop(['TARGET'], axis=1)
y_val = data_val.TARGET

# Treinamento 

In [37]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier

In [50]:
clf = RandomForestClassifier(max_depth=10)

In [51]:
clf.fit(x_train, y_train)

pred_train = clf.predict(x_train)

In [52]:
print(classification_report(pred_train, y_train))

              precision    recall  f1-score   support

      NORMAL       0.97      0.99      0.98      5279
 REABASTECER       0.98      0.93      0.96      2704

    accuracy                           0.97      7983
   macro avg       0.98      0.96      0.97      7983
weighted avg       0.97      0.97      0.97      7983



# Validação

In [53]:
pred_val = clf.predict(x_val)

In [54]:
print(classification_report(y_val, pred_val))

              precision    recall  f1-score   support

      NORMAL       0.92      0.85      0.88      1356
 REABASTECER       0.72      0.84      0.78       640

    accuracy                           0.85      1996
   macro avg       0.82      0.84      0.83      1996
weighted avg       0.86      0.85      0.85      1996



In [55]:
print(accuracy_score(y_val, pred_val))

0.845691382765531
